# Kernel regression

**What you will learn today**: You will learn how to implement kernel regression. First we analyze a synthetic dataset: you will build the feature map associated with different kernels and we see how the learning performance can be different. As always after having understood the theoretical insights we step on to real-dataset problem: you can play using our dear friend sklearn to implement Kernel Ridge Regression, with a kernel of your choice.

**Important note!** There are 2 evaluated questions! Make sure to answer them to get the points.

# 1) KRR from scratch 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Remember first lecture?  Let us use pandas for handling synthetic dataset

In [ ]:
data = pd.read_csv('dataTP8.csv')

In [ ]:
x,y = np.array(data['X']), np.array(data['Y'])

Let's see how it looks like

In [ ]:
plt.plot(x,y,'o') 

It does not seem a bad idea to use a polynomial fitting here. We have introduced in the theoretical lectures kernel methods: $$K(x,y) = {\phi(x)}^T{\phi(y)}$$

Let us pick for the first part of the analysis a feature map of the form:
$$ \phi_k^{(p)}(x) = x^k  \,\,\,\,\,\,\,\, k=1 \dots p $$

You may have studied this under the name *linear basis regression*.

The tools we need once we have mapped the datapoints in feature space are the usual one of linear regression as we know and all will be straight-forward.
We will study the behaviour as the degree $p$ varies, this will vary the complexity of the fitting model leading us to our usual bias-variance tradeoff consideration.

In [ ]:
def build_poly(x, degree,choice='none',gamma=1):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    x_pol = []
    
    for i in range(0,len(x)):
        x_pol_ele = []
        if choice == "rbf":
          x_pol_ele = np.exp(-x[i]*x[i]/gamma)*np.array([x[i]**j/(np.math.factorial(j)*np.math.sqrt(gamma**j)) for j in range(0,degree+1)])
        else:
          x_pol_ele = np.array([x[i]**j for j in range(0,degree+1)])
        x_pol.append(x_pol_ele)
    return np.array(x_pol)

def ridge_regression(y, X, lambda_):
    """implement ridge regression."""
    Xt = np.transpose(X)
    I = np.eye(len(Xt))
    w =  np.linalg.solve(Xt@X + (2*lambda_*len(y))*I,Xt@y)
    mse = (1/(2*len(y)))*np.sum((y-np.matmul(X,w))**2)
    return mse,w

def polynomial_regression(x,y,degrees,lambda_):
    num = len(degrees)
    fig, axs = plt.subplots(num,figsize=(15,15)) ; i =-1
    for degree in (degrees):
        i+=1   ; phi = build_poly(x, degree)
        
        mse , weights = ridge_regression(y, phi,lambda_)
        rmse = np.sqrt(2*mse)

        print("Processing {j}th experiment, degree={d}, rmse={loss}".format(
              j=i + 1, d=degree, loss=rmse))
        # print(weights)
        # plot fit
        # fhat = np.dot(phi,weights)
        xvals = np.arange(min(x) - 0.1, max(x) + 0.1, 0.1)
        phi = build_poly(xvals, degree) ; fhat = np.dot(phi,weights)
        axs[i].plot(x,
            y, 'o',label=f'Degree={degree} -- Data')
        axs[i].plot(xvals,
            fhat,label=f'Fit')
        axs[i].set_ylim([-1.4,1.4])
        axs[i].legend()
    # plt.tight_layout()
    

In [ ]:
degrees = [5,  12, 20,50,80,120,160] ; lambda_ = 10e-6
polynomial_regression(x,y,degrees,lambda_)

We clearly see that by choosing a value of the degree too high we start to have bad fitting (i.e. overfitting). This feature map is not building a good kernel for studying the data.

It is interesting to understand if we can build another feature map, always in a polynomial fashion, that is able to having good learning rates.

First let's introduce some functions to split data in training and testing dataset to evaluate the performance of the model.

In [ ]:
def split_data(x, y, ratio, seed=1):
    # set seed
    np.random.seed(seed)
    # ***************************************************
    index = np.arange(0,len(x))
    index_train = np.random.choice(index, int(ratio*len(x)), replace=False)
    
    training_x = x[index_train]
    training_y = y[index_train]
    test_x = x[[i for i in range(len(x)) if i not in index_train]]
    test_y = y[[i for i in range(len(y)) if i not in index_train]]
    
    return training_x, training_y, test_x, test_y

## Exercise
* Implement kernel regression with the feature map parametrized by $\gamma$ which we encode with the codework *rbf*: 

$$ \phi_k^{(p)}(x) = e^{\frac{-x^2}{\gamma}}\frac{x^k}{k!}  \,\,\,\,\,\,\,\, k=1 \dots p $$
* Study the behaviour as a function of $\lambda$ of the learning curves (test error and training error) and compare with what you would see with the previous feature map.
* What do you conclude?



Let us implement first the function for finding the learning curves of KRR.

In [ ]:
def ridge_regression_learning_curves(x, y, degree, ratio,lambdas, seed,choice , gamma):
    """ridge regression demo."""
    # define parameter
    training_x,training_y,test_x,test_y = split_data(x, y, ratio, seed)

    phi_train = build_poly(training_x, degree,choice,gamma)
    phi_test = build_poly(test_x, degree,choice,gamma)

    rmse_tr = []
    rmse_te = []
    for _, lambda_ in enumerate(lambdas):
        mse_train,w = ridge_regression(training_y, phi_train, lambda_)
        rmse_tr.append(np.sqrt(2*mse_train))
        
        mse_test = (1/(2*len(test_y)))*np.sum((test_y-np.matmul(phi_test,w))**2)
        rmse_te.append(np.sqrt(2*mse_test))
    
        # print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
        #        p=ratio, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))

    return rmse_tr , rmse_te

Let us start with the previous feature map, we already saw that the fitting was not good at all when the degree was too high (i.e. overfitting).

In [ ]:
seed = 1926  ; split_ratio = 0.5   ; choice = 'none'; lambdas = np.logspace(-6, 1, 100)
fig, ax = plt.subplots(len(degrees), figsize = (15,15)) 
for i,degree in enumerate(degrees):
  a,b = ridge_regression_learning_curves(x, y, degree, split_ratio, lambdas,seed,choice , 1)
  ax[i].plot(lambdas,a,label=f"Training, degree ={degree}") ;  ax[i].plot(lambdas,b,label=f"Test, degree={degree}") 
  ax[i].legend() ; ax[i].set_xscale('log') ; ax[i].set_yscale('log')

# 2) Kernel methods using Sklearn 

We have seen how to build Kernel Ridge Regression (KRR) from scratch starting from the feature map. The idea to map datapoint in an higher dimensional space (i.e. kernel trick) can be used also in classification task. We will consider again our old friend MNIST, we will consider SVC (Support Vector Classification). We will see that, at stake with previous lecture, *SVC* will accept the parameter *kernel* to choose in which space mapping our datapoints.

In [ ]:
import time

from sklearn.datasets import fetch_openml # MNIST data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

print(__doc__)

# Turn down for faster convergence
t0 = time.time()
train_size = 9000 # 60000
test_size = 1500 # 10000

### load MNIST data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', return_X_y=True, as_frame=False)
X = np.array(X)[:train_size+test_size]
y = np.array(y)[:train_size+test_size]

Let's plot an image to see how it looks like with plt.imshow.

In [ ]:
plt.imshow(X[6542,:].reshape(28,28))

We shuffle the data and we do the test-train splitting.

In [ ]:
# shuffle data
random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))
# pick training and test data sets 
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=train_size,test_size=test_size)

### Exercise [EVALUATED 1]
Use `sklearn.svm.SVC` to classify the MNIST dataset. Use the following kernels: `linear`, `poly`, `rbf`, `sigmoid`. For each kernel do a cross-validation to find the best hyperparameters. Use `GridSearchCV` to do that.
Finally return the best global model and the best kernel for the dataset.

# Backpropagation from scratch

**What will you learn today**: Backpropagation is the fundamental building block of deep learning, as it allows to automatically compute the gradients of any arbitrary compositional function with a similar computational cost than evaluating it. This makes possible to work with arbitrarily complex neural network architectures, composed of many layers, without the need to manually compute their gradients. Backpropagation is already implemented in all high-level deep learning frameworks, e.g. `PyTorch`, and as such, we would hardly ever need to think of how it works. However, it is a very educational exercise to implement it once in your life, and that is precisely what we will do in this exercise! In particular, you will learn to implement and derive the forward and backward pass of a very simple neural network in pure `numpy`. As a bonus, we will also explore how to approximate the non-convex loss landscape of a neural network by a convex one, and we will learn how to use such approximation to derive intuitions about how different design choices affect the network's behaviour.

# 1) Forward pass
To simplify the exercise, we will only work with a simple architecture, consisting of a feedforward neural network with two fully connected layers, i.e., a single-hidden layer MLP.

![simple_mlp](simple_mlp.png)

Mathematically, we can write the feedforward computation as:
$$ x_j^{(1)}=\sigma\left(z_j^{(1)}\right)=\sigma\left(\sum_{i=1}^D w_{i,j}^{(1)} x_i^{(0)}+b_j^{(1)}\right), $$
$$ \hat y =\sigma\left(z_1^{(2)}\right)=\sigma\left(\sum_{i=1}^K w_{i,1}^{(2)} x_i^{(1)}+b_1^{(2)}\right),  $$
where $\sigma(\cdot)$ denotes the sigmoid activation function. In the rest of the exercise, we will use $D=4$, and $K=5$.

We can alternatively write the same computation in vector notation
$$ \bf x^{(1)}=\sigma\left(\bf z^{(1)}\right)=\sigma\left(\bf W^{(1)} \bf x^{(0)}+\bf b^{(1)}\right), $$
$$ \hat y=\sigma\left(z^{(2)}\right)=\sigma\left(\bf {w^{(2)}}^\top \bf x^{(1)}+b^{(2)}\right). $$

In general, we will denote the function computed by the neural network as $f_{\bf w}(\bf x)=\hat y$, and use $\bf w$ to represent the vector of all weights in the architecture.

## Exercise


As a warm up, let's implement the activation function.

In [ ]:
import numpy as np

def sigmoid(t):
    """apply sigmoid function on t."""
    return 1.0 / (1 + np.exp(-t))

def grad_sigmoid(t):
    """return the derivative of sigmoid on t."""
    return sigmoid(t) * (1 - sigmoid(t))

We will initialize our data and parameters with some random numbers.

In [ ]:
x = np.array([0.01, 0.02, 0.03, 0.04])
W = {
    "w_1": np.ones((4, 5)),
    "w_2": np.ones(5)
}
y = 1

Then, let's implement the forward pass. If you implement it correctly, you should see that your code can pass the test successfully.

In [ ]:
def simple_feed_forward(x, W):
    """Do feed forward propagation.""" 
    x_0 = x
    z_1 = W["w_1"].T @ x_0
    x_1 = sigmoid(z_1)
    z_2 = W["w_2"].T @ x_1
    y_hat = sigmoid(z_2)
    
    return z_1, z_2, y_hat

try:
    expected = 0.93244675427215695
    _, _, yours = simple_feed_forward(x, W)
    assert np.sum((yours - expected) ** 2) < 1e-15
    print("Your implementation is correct!")
except:
    print("Your implementation is not correct.")

## 2) Backward pass

We now have a working implementation of our network! However, if we want to be able to train it using gradient descent, we need to be able to compute its gradient. Let's do that.

We will use the squared error as our loss function, i.e.,
$$\ell(y,\hat y)=\frac{1}{2}(\hat y-y)^2$$


### Exercise
Evaluate the derivative of $\mathcal{L}(\bf w)=\ell(y, f_{\bf w}(\bf x))$ with respect to $w_{i,1}^{(2)}$ and $w_{i,j}^{(1)}$ for a single training sample $(\bf x, y)$, by following the backpropagation algorithm.

### Exercise [EVALUATED 2]
Now that we have derived the backward pass analytically, let's implement it in Python!

*Hint*: You might want to slightly change `simple_feed_forward`.

In [ ]:
def simple_backpropagation(y, x, W):
    """Do backpropagation and get delta_W."""
    # Your code here
    pass
  
try:
    expected = {
        'w_1': np.array([
            [ -1.06113639e-05,  -1.06113639e-05,  -1.06113639e-05, -1.06113639e-05,  -1.06113639e-05],
            [ -2.12227277e-05,  -2.12227277e-05,  -2.12227277e-05, -2.12227277e-05,  -2.12227277e-05],
            [ -3.18340916e-05,  -3.18340916e-05,  -3.18340916e-05, -3.18340916e-05,  -3.18340916e-05],
            [ -4.24454555e-05,  -4.24454555e-05,  -4.24454555e-05, -4.24454555e-05,  -4.24454555e-05]]),
        'w_2': np.array(
            [-0.00223387, -0.00223387, -0.00223387, -0.00223387, -0.00223387])
    }
    yours = simple_backpropagation(y, x, W)
    assert np.sum(
        [np.sum((yours[key] - expected[key]) ** 2) for key in expected.keys()]) < 1e-15
    print("Your implementation is correct!")
except:
    print("Your implementation is not correct!")

# 3) Bonus: Effect of regularization

One of the first things we learn about neural networks is that their loss landscape is not convex. This means that analyzing how different design choices will affect their performance precisely is generally very hard. Fortunately, however, many times we can get an intuition of the behaviour of a neural network by taking a few approximations. We will now explore one of those. In particular, we will use some simple approximations to explore what is the effect of regularization on the weights of a neural network. 

Let $\bf w$ be the weight vetor of all weights in the neural network, and recall that we do not normally penalize the bias term, so let's ignore it for the rest of our derivations. Furthermore, let $\bf w^\star$ denote a parameter that minimizes the cost function $\mathcal L$ for the given test set (where the cost functions does not include the regularization). We would like to study how the optimal weight changes if we include some regularization.

In order to make the problem tractable, assume that $\mathcal L(\bf w)$ can be locally expanded around the optimal parrameter $\bf w^\star$ in the form
$$\mathcal L(\bf w) =\mathcal L(\bf w^\star)+\frac{1}{2}(\bf w-\bf w^\star)^\top\bf H(\bf w-\bf w^\star),$$
where $\bf H$ denotes the Hessian, whose components are the entries
$$\cfrac{\partial^2 \mathcal{L}}{\partial \bf w_i \partial \bf w_j }$$

Now, let's add a regularization term of the form $\frac{1}{2}\mu\|\bf w\|^2_2$.

## Exercise
1. Show that the optimum weight vector for the regularized problem is given by $$\bf Q(\bf \Lambda+\mu\bf I)^{-1}\bf \Lambda\bf Q^\top \bf w^\star$$ where $\bf H=\bf Q\bf\Lambda\bf Q^\top$ represents the eigenvalue decomposition of the symmetric matrix $\bf H$, i.e., $\bf Q$ is an orthonormal matrix, and $\bf \Lambda$ is a diagonal matrix whose entries are non-negative and decreasing along the diagonal.
2. Show that $(\bf\Lambda+\mu\bf I)^{-1}\bf\Lambda$ is again a diagonal matrix whose $i$-th entry is now $\lambda_i/(\lambda_i+\mu)$.
3. Argue that along the dimensions of the eigenvectors of $\bf H$ that correspond to large eigenvalues, essentially no changes occur in the weights, but that along the dimensions of eigenvectors of very small eigenvalues the weight is drastically decreased. 